## Imports

In [63]:
import os
from typing import List
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import sys
import logging 

logging.basicConfig(
     level=logging.INFO, 
     format= '[%(asctime)s|%(levelname)s|%(module)s.py:%(lineno)s] %(message)s',
     datefmt='%H:%M:%S'
 )
import tqdm.notebook as tq
from tqdm import tqdm
# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, 
    TrainingArguments, Trainer, EarlyStoppingCallback, IntervalStrategy
)

from defi_textmine_2025.data import TARGET_COL, INTERIM_DIR, MODELS_DIR, submission_path

In [20]:
RANDOM_SEED = 123  # random reproducibility
np.random.seed(RANDOM_SEED)
# BASE_CHECKPOINT = "bert-base-uncased"
BASE_CHECKPOINT = "bert-base-multilingual-cased"
# BASE_CHECKPOINT = "camembert/camembert-base"
TASK_NAME = "hasrelation_btc"
TASK_TARGET_COL = "label" # hasrelation?

entity_classes = {'TERRORIST_OR_CRIMINAL', 'LASTNAME', 'LENGTH', 'NATURAL_CAUSES_DEATH', 'COLOR', 'STRIKE', 'DRUG_OPERATION', 'HEIGHT', 'INTERGOVERNMENTAL_ORGANISATION', 'TRAFFICKING', 'NON_MILITARY_GOVERNMENT_ORGANISATION', 'TIME_MIN', 'DEMONSTRATION', 'TIME_EXACT', 'FIRE', 'QUANTITY_MIN', 'MATERIEL', 'GATHERING', 'PLACE', 'CRIMINAL_ARREST', 'CBRN_EVENT', 'ECONOMICAL_CRISIS', 'ACCIDENT', 'LONGITUDE', 'BOMBING', 'MATERIAL_REFERENCE', 'WIDTH', 'FIRSTNAME', 'MILITARY_ORGANISATION', 'CIVILIAN', 'QUANTITY_MAX', 'CATEGORY', 'POLITICAL_VIOLENCE', 'EPIDEMIC', 'TIME_MAX', 'TIME_FUZZY', 'NATURAL_EVENT', 'SUICIDE', 'CIVIL_WAR_OUTBREAK', 'POLLUTION', 'ILLEGAL_CIVIL_DEMONSTRATION', 'NATIONALITY', 'GROUP_OF_INDIVIDUALS', 'QUANTITY_FUZZY', 'RIOT', 'WEIGHT', 'THEFT', 'MILITARY', 'NON_GOVERNMENTAL_ORGANISATION', 'LATITUDE', 'COUP_D_ETAT', 'ELECTION', 'HOOLIGANISM_TROUBLEMAKING', 'QUANTITY_EXACT', 'AGITATING_TROUBLE_MAKING'}

generated_data_dir_path = os.path.join(INTERIM_DIR, "multilabel_tagged_text_dataset")
assert os.path.exists(generated_data_dir_path)
train_dir = os.path.join(generated_data_dir_path, "train")
test_dir = os.path.join(generated_data_dir_path, "test")

preprocessed_data_dir = os.path.join(INTERIM_DIR, "one_hot_multilabel_tagged_text_dataset")
labeled_preprocessed_data_dir_path = os.path.join(preprocessed_data_dir,"train")
! mkdir -p {labeled_preprocessed_data_dir_path}

model_dir_path = os.path.join(MODELS_DIR, f"finetuned-{BASE_CHECKPOINT}")
! mkdir -p {model_dir_path}
model_dict_state_path = os.path.join(model_dir_path,"MLTC_model_state.bin")


In [21]:
def get_cat_var_distribution(cat_var: pd.Series) -> pd.DataFrame:
    return pd.concat(
        [cat_var.value_counts(), cat_var.value_counts(normalize=True)], axis=1
    )

## Prepare the datasets for the binary text classification

### Load and process the target

In [22]:
def load_csv(dir_or_file_path: str, index_col=None, sep=',') -> pd.DataFrame:
    if os.path.isdir(dir_or_file_path):
        all_files = glob.glob(os.path.join(dir_or_file_path , "*.csv"))  
    else:
        assert dir_or_file_path.endswith(".csv")
        all_files = [dir_or_file_path]
    assert len(all_files) > 0
    return pd.concat([pd.read_csv(filename, index_col=index_col, header=0, sep=sep) for filename in all_files], axis=0, ignore_index=True)

labeled_df = load_csv(train_dir).assign(**{TASK_TARGET_COL: lambda df: 2 + ~pd.isnull(df.relations).astype(int)}).drop(TARGET_COL, axis=1)
labeled_df

,text_index,e1,e2,text,label
0,2576,0,0,"Le matin du 10 janvier 2010, Arthur et Jacques...",0
1,2576,1,0,"Le matin du 10 janvier 2010, Arthur et Jacques...",0
2,2576,0,1,"Le matin du 10 janvier 2010, Arthur et Jacques...",1
3,2576,1,1,"Le matin du 10 janvier 2010, Arthur et Jacques...",0
4,2576,2,0,"Le matin du 10 janvier 2010, Arthur et Jacques...",0
...,...,...,...,...,...
124000,41884,11,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",0
124001,41884,12,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",0
124002,41884,15,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",0
124003,41884,17,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",1


In [23]:
get_cat_var_distribution(labeled_df[TASK_TARGET_COL])

,count,proportion
label,,
0,97611,0.787154
1,26394,0.212846


### train-validation split

In [25]:
VALIDATION_RATE = 0.2
train_df, val_df = train_test_split(labeled_df, test_size=VALIDATION_RATE, shuffle=True, random_state=RANDOM_SEED, stratify=labeled_df[TASK_TARGET_COL])
train_df.shape, val_df.shape

((99204, 5), (24801, 5))

In [26]:
pd.concat([get_cat_var_distribution(train_df[TASK_TARGET_COL]), compute_class_weights(train_df)], axis=1)

,count,proportion,weight
0,78089,0.787156,0.793998
1,21115,0.212844,2.936420


In [27]:
get_cat_var_distribution(val_df[TASK_TARGET_COL])

,count,proportion
label,,
0,19522,0.787146
1,5279,0.212854


## Create the tokenized datasets for model input

### init the tokenizer

In [28]:
# Hyperparameters
tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT)
task_special_tokens = ["<e1>", "</e1>", "<e2>", "</e2>"] + [
    f"<{entity_class}>" for entity_class in entity_classes
]
# add special tokens to the tokenizer
num_added_tokens = tokenizer.add_tokens(task_special_tokens, special_tokens=True)
num_added_tokens, len(tokenizer)

/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(59, 119606)

### init the train-valid datasets from dataframe

In [29]:
train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(train_df, preserve_index=False)
task_datasets = DatasetDict({"train": train_ds, "validation": val_ds})
task_datasets

DatasetDict({
    train: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label'],
        num_rows: 99204
    })
    validation: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label'],
        num_rows: 99204
    })
})

In [34]:
task_datasets["train"][0]

{'text_index': 51469,
 'e1': 14,
 'e2': 6,
 'text': 'La présence à forte dose d’une substance toxique a été retrouvée dans un stock de raisins blancs destinés à la fabrication de vin dans le <e1><PLACE>sud</e1> de l’Italie. Selon l’œnologue M. Djael Rodriguez, ce fait est de plus en plus courant dans l’univers viticole de cette <e1><PLACE>partie</e1> du pays où pourtant, aucune <e2><MATERIEL>substance chimique toxique</e2> n’est introduite dans le sol. C’est la preuve pour lui que tout est lié, et qu’il faudrait que le Ministère de l’Eau et de l’Assainissement assainisse le réseau hydraulique du pays dans son ensemble. Une telle mesure évitera, à l’avenir, que toute une récolte ne soit détruite à l’incinérateur, ce qui constitue un important manque à gagner pour les propriétaires de vignobles, souvent endettés à cause du fort coût des équipements et des frais fonciers.',
 'label': 0}

In [33]:
task_datasets["train"][1]

{'text_index': 51407,
 'e1': 0,
 'e2': 18,
 'text': "Le <e2><TIME_EXACT>12 mai 2016</e2>, un <e1><NATURAL_EVENT>tremblement de terre</e1> a frappé de plein fouet l'Arizona. Plusieurs personnes ont dû quitter leur domicile grâce à des camions et des hélicoptères mis à disposition par le gouvernement. Le <e1><NATURAL_EVENT>sinistre</e1> a provoqué la destruction de plusieurs bâtiments, dont des hôpitaux. Plusieurs patients ont dû être évacués en urgence. Parmi eux, il y avait Madame Belkiss Miller, une femme de 21 ans qui avait donné naissance à un bébé prématuré dans une ambulance à l'entrée de la ville de Phoenix. Les pompiers se sont servis d'un brancard et de serviettes qu'ils avaient sous la main pour aider la patiente. La jeune femme et son fils s'en sont sortis sains et saufs. Les opérations de sauvetage ont continué dans tout l'État.",
 'label': 1}

### Tokenize the datasets

In [53]:
def tokenize_function(example: dict):
    return tokenizer(example["text"], truncation=True)

# We’re using batched=True in our call to map so the function is applied to multiple elements of our dataset at once, and not on each element separately
# This is way faster
# Source https://huggingface.co/learn/nlp-course/chapter3/2?fw=pt
tokenized_datasets = task_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/99204 [00:00<?, ? examples/s]

Map:   0%|          | 0/99204 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 99204
    })
    validation: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 99204
    })
})

In [54]:
tokenized_datasets.column_names

{'train': ['text_index',
  'e1',
  'e2',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'validation': ['text_index',
  'e1',
  'e2',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask']}

### Test the batch-level padding with a data collator

In [55]:
# columns are removed because DataCollatorWithPadding doesn't support any other columns than the ones produced by the tokenizer (or non tensors)
samples = tokenized_datasets.remove_columns(task_datasets["train"].column_names)["train"][:8]
samples = {k: v for k, v in samples.items()}
[len(x) for x in samples["input_ids"]]

[209, 196, 233, 197, 192, 153, 166, 187]

In [56]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 233]),
 'token_type_ids': torch.Size([8, 233]),
 'attention_mask': torch.Size([8, 233])}

## Fine-tuning a model with the Trainer API

### Compute the weight of classes to handle imbalance

In [ ]:
# Source: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#calculate_class_weights
# Scaling by total/2 helps keep the loss to a similar magnitude.
n_examples = labeled_df.shape[0]
n_classes = labeled_df[TASK_TARGET_COL].nunique()
def compute_class_weights(lbl_df: pd.DataFrame) -> pd.Series:
    return get_cat_var_distribution(lbl_df[TASK_TARGET_COL]).reset_index(drop=False)["count"].apply(lambda x: (1 / x) * (n_examples / n_classes)).rename("weight")
pd.concat([get_cat_var_distribution(labeled_df[TASK_TARGET_COL]), compute_class_weights(labeled_df)], axis=1)

,count,proportion,weight
0,97611,0.787154,0.635200
1,26394,0.212846,2.349113


### Init the model

In [58]:
model = AutoModelForSequenceClassification.from_pretrained(BASE_CHECKPOINT, num_labels=n_classes)

/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Init the trainer

Source: https://stackoverflow.com/questions/69087044/early-stopping-in-bert-trainer-instances#69087153

1. Use `load_best_model_at_end = True` (EarlyStoppingCallback() requires this to be True).
2. `evaluation_strategy = 'steps'` or IntervalStrategy.STEPS instead of 'epoch'.
3. `eval_steps = 50` (evaluate the metrics after N steps).
4. `metric_for_best_model = 'f1'`

In [ ]:
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="macro")
    precision = precision_score(y_true=labels, y_pred=pred, average="macro")
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir=os.path.join(MODELS_DIR, f"{TASK_NAME}-trainer"),
    per_device_train_batch_size=16,    
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    eval_strategy=IntervalStrategy.STEPS, # steps
    eval_steps = 50, # Evaluation and Save happens every 50 steps
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=os.path.join(MODELS_DIR, f"{TASK_NAME}-trainer-tensorboard"),
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
    save_total_limit = 2, # Only last 2 models are saved. Older ones are deleted
    push_to_hub=False,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

### Launch the training

In [ ]:
trainer.train()